## my_kmeans 함수

In [57]:
def my_kmeans(test, k, random_state=None):
    import numpy as np
    import pandas as pd
    test2 = np.array(test.copy())
    test2 = pd.DataFrame(test2)
    np.random.seed(random_state)
    temp_cluster = test2.iloc[np.random.choice(len(test2), k, replace=False), :]
    cluster = temp_cluster.copy() #SettingWithCopyWarning를 없애기 위해서...
    cluster['kind'] = np.random.choice(k, k, replace=False)
    a = pd.DataFrame({'kind':[999]*len(test2), 'd':[9999999]*len(test2)})
    a = pd.concat([test2, a], axis=1)
    while True:
        t_cluster = cluster.copy() #반복문을 끝내는 조건을 만들기위해
        for i in cluster['kind']:
            #행렬연산(넘파이)를 이용한 거리계산
            temp = (test2.values - cluster[cluster['kind']==i].iloc[:, :-1].values)**2
            temp = np.sqrt(np.sum(temp, axis=1))
            ###SettingWithCopyWarning를 없애기 위한과정 시작###
            ak = []
            ad = []
            for j in range(len(temp)):
                if a['d'][j] > temp[j]:
                    ak.append(i)
                    ad.append(temp[j])
                else:
                    ak.append(a['kind'][j])
                    ad.append(a['d'][j])
            a['kind'] = ak
            a['d'] = ad
            ###SettingWithCopyWarning를 없애기 위한과정 끝###
            #데이터프레임의 일부 컬럼의 데이터만 변경하기 위한 방법
            c = cluster[cluster['kind']==i].copy()
            c.iloc[:,:-1] = np.mean(a[a['kind']==i].iloc[:,:-2])
            cluster[cluster['kind']==i] = c
        if np.sum(np.abs(t_cluster-cluster)).sum()==0: #반복문 끝내는 조건
            return np.array(a['kind'])
            break

## 유방암 데이터로 사이킷런의 KMeans와 성능 비교

### 데이터 준비

In [58]:
import pandas as pd
wisc = pd.read_csv("wisc_bc_data.csv")
x = wisc.iloc[:, 2:]
y = wisc['diagnosis']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x)
x2 = scaler.transform(x)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x2, y, test_size=0.9, random_state=1)

### 모델 생성 및 예측

In [70]:
from sklearn.cluster import KMeans
model_sk = KMeans(n_clusters = 2, random_state=1)
model_sk.fit(x_train, y_train)
result_sk = model_sk.predict(x_test)
result_my = my_kmeans(x_test, 2, random_state=1)

### 성능 비교

In [71]:
print('####### 사이킷런 kmeans #######')
print(pd.crosstab(result_sk, y_test))
print('####### my kmeans #######')
print(pd.crosstab(result_my, y_test))

####### 사이킷런 kmeans #######
diagnosis    B    M
row_0              
0          321   37
1            3  152
####### my kmeans #######
diagnosis    B    M
row_0              
0          113    4
1          211  185


### 성능 비교 결과
* 성능 비교 결과
    1. 사이킷런의 분류결과가 더 좋게 나왔다.
    2. 사이킷런은 씨드값(random_state)에 따른 변화가 없었으나,
        만든 kmeans는 씨드값에 따라서 분류결과의 격차가 컸다.
* 검토 및 개선 방향
    1. my_kmeans는 처음 클러스터 중심이 완전 랜덤이다보니 씨드값에 따른 변화가 크다.
    2. 그러므로 처음 클러스터 중심을 랜덤이 아닌 데이터의 중앙값으로 설정해서
        분류에 좀 더 특화된 옵션을 추가하자.

In [64]:
def my_kmeans(test, k, random_state=None, init='random'):
    import numpy as np
    import pandas as pd
    test2 = np.array(test.copy())
    test2 = pd.DataFrame(test2)
    np.random.seed(random_state)
    #####초기 클러스터 중앙 설정 코드 시작#####
    if init == 'random':
        temp_cluster = test2.iloc[np.random.choice(len(test2), k, replace=False), :]
        cluster = temp_cluster.copy() #SettingWithCopyWarning를 없애기 위해서...
    elif init == 'median':
        cluster=[]
        for j in range(k):
            c=[]
            for i in test2.columns:
                m = (test2[i][pd.qcut(test2[i], k, labels=list(range(k)))==j]).median()
                c.append(m)
            cluster.append(c)
        cluster = pd.DataFrame(cluster)
    elif init == 'mean':
        cluster=[]
        for j in range(k):
            c=[]
            for i in test2.columns:
                m = (test2[i][pd.qcut(test2[i], k, labels=list(range(k)))==j]).mean()
                c.append(m)
            cluster.append(c)
        cluster = pd.DataFrame(cluster)
    #####초기 클러스터 중앙 설정 코드 끝#####   
    cluster['kind'] = np.random.choice(k, k, replace=False)
    a = pd.DataFrame({'kind':[999]*len(test2), 'd':[9999999]*len(test2)})
    a = pd.concat([test2, a], axis=1)
    while True:
        t_cluster = cluster.copy() #반복문을 끝내는 조건을 만들기위해
        for i in cluster['kind']:
            #행렬연산(넘파이)를 이용한 거리계산
            temp = (test2.values - cluster[cluster['kind']==i].iloc[:, :-1].values)**2
            temp = np.sqrt(np.sum(temp, axis=1))
            #####SettingWithCopyWarning를 없애기 위한 방법 코드 시작#####
            ak = []
            ad = []
            for j in range(len(temp)):
                if a['d'][j] > temp[j]:
                    ak.append(i)
                    ad.append(temp[j])
                else:
                    ak.append(a['kind'][j])
                    ad.append(a['d'][j])
            a['kind'] = ak
            a['d'] = ad
            #####SettingWithCopyWarning를 없애기 위한 방법 코드 끝#####
            #데이터프레임의 일부 컬럼의 데이터만 변경하기 위한 방법
            c = cluster[cluster['kind']==i].copy()
            c.iloc[:,:-1] = np.mean(a[a['kind']==i].iloc[:,:-2])
            cluster[cluster['kind']==i] = c
        if np.sum(np.abs(t_cluster-cluster)).sum()==0: #반복문 끝내는 조건
            return np.array(a['kind'])
            break

### 함수 개선 후 성능 비교

In [69]:
from sklearn.cluster import KMeans
model_sk = KMeans(n_clusters = 2, random_state=1)
model_sk.fit(x_train, y_train)
result_sk = model_sk.predict(x_test)
result_my = my_kmeans(x_test, 2, random_state=1, init='median')
print('####### 사이킷런 kmeans #######')
print(pd.crosstab(result_sk, y_test))
print('####### my kmeans #######')
print(pd.crosstab(result_my, y_test))

####### 사이킷런 kmeans #######
diagnosis    B    M
row_0              
0          321   37
1            3  152
####### my kmeans #######
diagnosis    B    M
row_0              
0          311   35
1           13  154
